<a href="https://colab.research.google.com/github/Tristan07999/TensorFlow/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")



In [26]:
california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0 
california_housing_dataframe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
3579,-117.9,33.9,22.0,1178.0,289.0,865.0,294.0,3.0,180.0
15378,-122.3,37.5,39.0,1459.0,285.0,761.0,291.0,5.0,298.1
55,-115.5,33.0,10.0,1879.0,387.0,1376.0,337.0,2.0,67.5
10915,-120.8,37.5,52.0,1724.0,352.0,922.0,348.0,1.7,85.7
4241,-118.0,33.9,34.0,1932.0,315.0,1053.0,316.0,5.1,213.3
...,...,...,...,...,...,...,...,...,...
11525,-121.3,37.9,43.0,1610.0,412.0,1409.0,365.0,1.8,51.7
4855,-118.1,34.6,29.0,2843.0,603.0,1517.0,573.0,2.7,106.9
10049,-119.8,36.8,52.0,1185.0,260.0,635.0,239.0,1.2,56.1
12074,-121.4,37.7,2.0,2682.0,393.0,883.0,271.0,6.0,196.7


In [27]:
california_housing_dataframe.describe()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0,17000.0
mean,-119.6,35.6,28.6,2643.7,539.4,1429.6,501.2,3.9,207.3
std,2.0,2.1,12.6,2179.9,421.5,1147.9,384.5,1.9,116.0
min,-124.3,32.5,1.0,2.0,1.0,3.0,1.0,0.5,15.0
25%,-121.8,33.9,18.0,1462.0,297.0,790.0,282.0,2.6,119.4
50%,-118.5,34.2,29.0,2127.0,434.0,1167.0,409.0,3.5,180.4
75%,-118.0,37.7,37.0,3151.2,648.2,1721.0,605.2,4.8,265.0
max,-114.3,42.0,52.0,37937.0,6445.0,35682.0,6082.0,15.0,500.0


**Step 1: Define features; configure feature columns; Define the target**


In [0]:
#Define input feature: total_rooms
my_feature = california_housing_dataframe[["total_rooms"]]

#configure a numeric feature column for total_rooms
feature_columns = [tf.feature_column.numeric_column("total_rooms")]

#define the target (lable)
targets = california_housing_dataframe["median_house_value"]



**Step 3: Configure the LinearRegressor**

In [0]:
#Configure a linear regression model using LinearRegressor
#Use gradient descent as the optimizer for training the model
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

#Configure the linear regression model with our feature columns and optimizer
linear_regressor = tf.estimator.LinearRegressor(feature_columns = feature_columns, optimizer = my_optimizer)

**Step 4: Define the input function**

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

**Step 5: Train the model**

In [0]:
_ = linear_regressor.train(input_fn = lambda:my_input_fn(my_feature, targets), steps = 100)

**Step 6: Evaluate the Model**